### TDCJ Web Scraper

In [220]:
# import libraries
from bs4 import BeautifulSoup
import urllib.request as url, time, pymysql.cursors

<b>Set connection to database</b>

In [221]:
# Connect to the database
connection = pymysql.connect(host='host.com',
                             user='username',
                             password='password',
                             db='database_name_here',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

<b>Page Scrape Mapping to Database</b>

In [222]:
scrapeToDbMap = {'SID Number:': 'sid',
'TDCJ Number:': 'tdcj',
'Name:': 'name',
'Race:': 'race',
'Gender:': 'gender',
'DOB:': 'dob',
'Projected Release Date:': 'projected_release_date',
'Parole Eligibility Date:': 'parole_elig_date',
'Offender Visitation Eligible:': 'offender_visitation_elig'}

<b>Function for scraping, parsing and recording data</b>

In [223]:
def tdcjScraper(webPage, sid):
    indexPlace = 0 # used for index reference
    titleVar = ""  # holder for variable values as iteration takes place
    valueList = [] # stores iterated variable values
    variableList = ('SID Number:','TDCJ Number:','Name:','Race:','Gender:','DOB:','Maximum Sentence Date:'
               'Current Facility:','Projected Release Date:','Parole Eligibility Date:',
                'Offender Visitation Eligible:')
    
    #store the page you visited in MySQL
    try:
        with connection.cursor() as cursor:
            # Create a new record
            sql = "INSERT INTO `tdcj_offenders` (sid) VALUES ('%s')" % (sid)
            cursor.execute(sql)
            # print (sql)
        # to save, commit your changes.
        connection.commit()
    except:
        pass
    page = url.urlopen(webPage) # open URL & store in page variable
    # parse the html using beautiful soup and store in variable `soup`
    soup = BeautifulSoup(page, 'html.parser')
    inmateData = soup.find(id='content_right') # select area to grab data from
    
    for i in inmateData.stripped_strings:
        valueList.append(i)
        if titleVar in variableList:
            if i in variableList:
                break # if it's an empty record, move on
            else:
                try:
                    with connection.cursor() as cursor:
                        # Create a new record
                        sql = """UPDATE `tdcj_offenders` SET %s = '%s' WHERE `sid` = '%s'""" % (scrapeToDbMap[titleVar], 
                                                                                                 valueList[indexPlace], 
                                                                                                 sid)
                        cursor.execute(sql)
                        # commit to save your changes.
                        # print (titleVar,valueList[indexPlace])
                        connection.commit()
                except:
                    pass
        titleVar = i
        indexPlace += 1

<b>Iterate through and SID range</b>

In [ ]:
for i in range(5138054, 5140000):
    sid = format(i, '08')
    tdcjPage = """https://offender.tdcj.texas.gov/OffenderSearch/offenderDetail.action?sid=%s""" % (sid) 
    time.sleep(2)
    # print (tdcjPage)
    tdcjScraper(tdcjPage, sid)

In [ ]:
connection.close()

<b>SQL to create the table the code will be writing to<b/>

```sql
-- DDL Code for the table holding the results of the web scraping
CREATE TABLE `tdcj_offenders` (
  `sid` varchar(15) COLLATE utf8_unicode_ci DEFAULT NULL,
  `tdcj` varchar(15) COLLATE utf8_unicode_ci DEFAULT NULL,
  `name` varchar(255) COLLATE utf8_unicode_ci DEFAULT NULL,
  `race` varchar(20) COLLATE utf8_unicode_ci DEFAULT NULL,
  `gender` varchar(20) COLLATE utf8_unicode_ci DEFAULT NULL,
  `dob` varchar(50) COLLATE utf8_unicode_ci DEFAULT NULL,
  `projected_release_date` varchar(50) COLLATE utf8_unicode_ci DEFAULT NULL,
  `parole_elig_date` varchar(50) COLLATE utf8_unicode_ci DEFAULT NULL,
  `offender_visitation_elig` varchar(50) COLLATE utf8_unicode_ci DEFAULT NULL,
  KEY `sid` (`sid`)
) ENGINE=MyISAM DEFAULT CHARSET=utf8 COLLATE=utf8_unicode_ci COMMENT='holds information regarding offenders registered in tdcj site'
```